## 트리의 앙상블

In [1]:
# 정형데이터에 가장 뛰어난 성능을 보이는 모델들 입니다.
# 앙상블 모델들은 결정트리(Decision Tree)를 기반으로 만들어졌습니다.
# 앙상블 모델들..
# - 랜덤포레스트(Rendom Forest)
# - 엑스트라 트리(Extra Trees)
# - 그레디언트 부스팅(Gradient Boosting)
# - 히스토그램 기반 그레디언트 부스팅(Histogram-base Gradient Booting)

## 랜덤포레스트(Random Forest)


In [2]:
### 앙상블 모델 중에 가장 대표격 모델로 사용됨
# - 안정적인 성능으로 널리 사용됨
# - 앙상블 모델 중에 가장 먼저 시도하는 모델이라고 보면 됩니다.
# - 훈련데이터에서 과대적합되는 것을 막아줍니다.
# - 검증데이터와 테스트데이터에서 안정적인 성능을 얻을 수 있음

### 학습 개념
# - 결정트리를 랜덤하게 만들어서 숲을 만든다고 보시면 됩니다.
# - 훈련데이터에서 랜덤하게 샘플을 추출하여 훈련을 완료한 후 
#   다시 반환을 합니다.
# - 랜덤하게 추출 시 이전에 사용된 샘플을 사용할 수도 있음
# (중복을 허용함)

### 부트스트랩 샘플
# - 위에 설명한 랜덤한 샘플 추출 시 중복을 허용하여 데이터를 샘플링 하는 방식
# - 샘플 추출 방식
# 1. 원본에서 랜덤 샘플 추출
# 2. 훈련 이후 사용이 끝나면 원본에 반환
# 3. 다시 원본에서 샘플 추출, 이때 중복 값 추출 될 수도 있음
#   위 순서를 반복하면서 샘플링을 통해 훈련하는 방식을 랜덤포레스트가 적용하고 있음

### *** 랜덤포레스트는 교차검증을 허용 합니다. ***



In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [3]:
wine = pd.read_csv('./data/08_wine.csv')


data = wine[['alcohol','sugar','pH']].to_numpy()
target = wine['class'].to_numpy()

train_input,test_input,train_target,test_target = train_test_split(data,target, random_state=42)

print(train_input.shape, train_target.shape)
print(test_input.shape, test_target.shape)

(4872, 3) (4872,)
(1625, 3) (1625,)


## 훈련모델 생성하기

In [13]:
# 랜덤포레스트 클래스(모델) : RandomForestClassifier
# 교차검증 : cross_validate()
# 교차검증 후 훈련검증결과와 테스트검증결과 확인하기
from sklearn.ensemble import RandomForestClassifier 
from sklearn.model_selection import cross_validate

#랜덤포레스트 객체생성 : 코어 모두 사용
rf = RandomForestClassifier(n_jobs=-1, random_state=42)

#교차검증 진행
# - return_train_score : 검증결과 반환받기
scores = cross_validate(rf,train_input, train_target,return_train_score=True,n_jobs=-1)


scores
#fit_time = 훈련시간
#score_time = 검증시간
#test_time = 테스트 시간
rf.fit(train_input,train_target)

RandomForestClassifier(n_jobs=-1, random_state=42)

In [11]:
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.997844759088341 0.8914208392565683


In [14]:
## 특성 중요도 조회하기
print(rf.feature_importances_)
# 랜덤포레스트는 독립변수를 골고루 쓴다(일반화시키는데 도움을 많이 주는 모델이다)

[0.23155241 0.49706658 0.27138101]


In [20]:
###  oob 기능 사용
# 훈련에 참여하지 못한 잠여 샘플 사용하는 기능
# 기본은 사용안함,
rf = RandomForestClassifier(oob_score =True,n_jobs = -1, random_state = 42)
rf.fit(train_input,train_target)
print(rf.oob_score_)

0.8981937602627258


## 엑스트라 트리

In [ ]:
# - 랜덤포레스트와 유사하게 작동
# - 기본적으로 100개의 결정트리를 훈련함
# - 랜덤포레스트와의 차이점
# : 부트스트랩 샘플링을 지원하지 않음
# : 훈련데이터 전체를 이용하여 결정트리를 생성
# : 무작위로 트리를 분리함
# - 사용되는 속성 : splitter = 'random' 무작위 속성
# - 장점
# : 과대적합을 막고, 검증데이터의 평가 값을 높일 수 있음
# :특성 데이터가 많지 않은 경우에는 랜덤포레스트와 큰 차이가 없음
# - 랜덤포레스트는 불순도 등 여러가지 조건에 따라 결정트리를 생성하기 때문에
# 속도가 느린 반면에 ,
# - 엑스트라트리는 랜덤하게 트리를 생성하기에 속도가 다소 빠르다는 장점이 있음

In [ ]:
###사용패키지 : 랜덤포레스트와 동일
# 사용되는 클래스(모델) : ExtraTreesClassifier

### 코어 전체사용, train 및 test 결과값 출력..
### 최종 결과인 train 및 test 결과 확인해 주세요.

In [23]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import cross_validate

et = ExtraTreesClassifier(n_jobs=-1,random_state=42)
scores = cross_validate(et, train_input, train_target,
                        return_train_score = True, n_jobs = -1)

#최종 훈련평가 결과 및 검증 결과
scores
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

0.997844759088341 0.8903937240035804


In [26]:
et.fit(train_input,train_target)
print(et.feature_importances_)
# [0.23155241 0.49706658 0.27138101] (랜덤포레스트)

[0.20702369 0.51313261 0.2798437 ]


## 그레디언트 부스팅(Gradient Boosting)

In [ ]:
# 깊이(max_depth)가 얕은 결정트리를 사용함
# - 기본적으로 amx_depth=3을 사용
# - 결정트리는 100개 사용
### **** 기존에 다른 훈련모델의 결과가 좋지 않을 때 사용하는 모델 ****
# 기존 훈련모델의 오차를 많이 보완해 줍니다.
#성능 향상을 위한 모델로 주로 사용됩니다.
#과대적합에 강하며, 일반화(과대/과소적합이 없는 상태)에 강합니다.

#성능 향상 테스트 방법
# - 결정트리의 갯수를 조절하면서 테스트 진행
# - 학습률을 지원하기 때문에 학습률의 값을 증가시키면서 테스트 진행
#  :기본 학습률은 0.1

#단점
# - 순서대로 트리를 추가(랜덤하지 않음) 하지 않기 때문에
# - 이런 느린 속도를 개선한 모델임
#  "히스토그램 기반 그레디언트 부스팅' 모델임



## 그레디언트 부스팅 모델 생성

In [27]:
### 사용하는 클래스(모델) : GradientBoostingClassifier
# 객체 생성시 아무것도 안주고 seed값만 줍니다.
# 교차 검증시에는 train, test 결과값 출력 합니다.

from sklearn.ensemble import GradientBoostingClassifier
gb=GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb,train_input,train_target,
                        return_train_score=True,n_jobs=-1)

scores
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

0.8894704231708938 0.8715107671247301


In [30]:
gb.fit(train_input,train_target)
print(gb.feature_importances_)

[0.12517641 0.73300095 0.14182264]


## 학습률 적용하기

In [33]:
# 학습률이 커지면 트리 보정을 강하게 하기 때문에,
# 복잡한 모델을 만들어서 일반화 성능을 떨어뜨리게 된다.
# 학습률 : learning rate =0.1 기본값..

#learnin_rate는 가급적 안건들이는게 좋다(건들이면 과대적합이 잘 일어남)

from sklearn.ensemble import GradientBoostingClassifier
gd=GradientBoostingClassifier(n_estimators=100,
                              learning_rate = 0.1,
                              random_state=42)
scores = cross_validate(gd,train_input,train_target,
                        return_train_score=True,n_jobs=-1)

scores
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

0.8894704231708938 0.8715107671247301


## 히스토그램 기반 그레디언트 부스팅
##### - Histogram-base Gradient Boosting

In [36]:
## 사용하는 클래스(모델) : HistogramBoostingClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
hgb=HistGradientBoostingClassifier(random_state=42)

scores = cross_validate(hgb, train_input,train_target,
                        return_train_score =True,n_jobs=-1)

scores
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

0.9380129799494501 0.8805410414363187


In [38]:
hgb.fit(train_input,train_target)
hgb.score(test_input,test_target)

0.8584615384615385

## 사이킷런 이외 다른 패키지에서 지원하는
## 히스토그램 기반 그레디언트 부스팅 기능 모델들

### XGBoost

In [43]:
from xgboost import XGBClassifier

xgb = XGBClassifier(tree_method='hist', random_state=42)
scores = cross_validate(xgb, train_input,train_target,
                        return_train_score =True,n_jobs=-1)

print(np.mean(scores['train_score']),np.mean(scores['test_score']))

0.9614122399872658 0.8834151529510873


### LightGBM
##### - 마이크로소프트에서 만든 히스토그램 기반 그레디언트 부스트 패키지
##### - 훈련 속도가 매우 빠름
##### - 최신 기술을 많이 적용하고 있어서, 인기가 올라가고 있음


In [45]:
from lightgbm import LGBMClassifier

In [47]:
lgb=LGBMClassifier()

scores = cross_validate(lgb, train_input,train_target,
                        return_train_score =True,n_jobs=-1)

scores
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

0.9413484712095832 0.8846461327857632


In [50]:
# [문제] 와인 데이터 사용
# 와인의 화학 조성을 사용하여 와인의 종류 예측 (자유롭게)..

# ** 특성 이름을 담고 있는 key 값 = feature_names
# ** 특성 데이터를 담고 있는 key 값 = data
# ** 범주 와인의 종류를 담고 있는 key 값 = target_names
#    -범주는 'class_0'과 'class_1'만 사용 (0과 1로 변경하여 사용)
#    -(0 =레드와인, 1 = 화이트와인)

#알콜(ALcohol)
#말산(Malic acid)
#회분(Ash)
#회분의 알칼리도(Alcalinity of ash)
#마그네슘(Magnesium)
#총 폴리페놀(Total phenols)
#플라보노이드 폴리페놀(Flavanoids)
#비 플라보노이드 폴리페놀(Nonflavanoid phenols)
#프로안토시아닌(Proanthocyanins)
#색상의 강도(Color intensity)
#색상(Hue)
#희석 와인의 OD280/OD315 비율( OD280/OD315 of diluted wines)
#프롤린(Proline)
from sklearn.datasets import load_wine
wine_all = load_wine()


{'data': array([[1.423e+01, 1.710e+00, 2.430e+00, ..., 1.040e+00, 3.920e+00,
         1.065e+03],
        [1.320e+01, 1.780e+00, 2.140e+00, ..., 1.050e+00, 3.400e+00,
         1.050e+03],
        [1.316e+01, 2.360e+00, 2.670e+00, ..., 1.030e+00, 3.170e+00,
         1.185e+03],
        ...,
        [1.327e+01, 4.280e+00, 2.260e+00, ..., 5.900e-01, 1.560e+00,
         8.350e+02],
        [1.317e+01, 2.590e+00, 2.370e+00, ..., 6.000e-01, 1.620e+00,
         8.400e+02],
        [1.413e+01, 4.100e+00, 2.740e+00, ..., 6.100e-01, 1.600e+00,
         5.600e+02]]),
 'target': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [51]:
wine_input = np.array(wine_all['data'][0:130])
wine_target = np.array(wine_all['target'][0:130])

In [52]:
train_input, test_input, train_target, test_target = train_test_split(wine_input,wine_target,test_size = 0.25,random_state=42)

In [53]:
from sklearn.ensemble import RandomForestClassifier 
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

from sklearn.model_selection import cross_validate

In [68]:
rf = RandomForestClassifier(n_jobs=-1, random_state=42)
et = ExtraTreesClassifier(n_jobs=-1,random_state=42)
gb=GradientBoostingClassifier(random_state=42)
hgb=HistGradientBoostingClassifier(random_state=42)
xgb = XGBClassifier(tree_method='hist', random_state=42)
lgb=LGBMClassifier()



scores_rf = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)
scores_et = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)
scores_gb = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)
scores_hgb= cross_validate(hgb,train_input, train_target, return_train_score=True, n_jobs=-1)
scores_xgb= cross_validate(xgb,train_input, train_target, return_train_score=True, n_jobs=-1)
scores_lgb= cross_validate(lgb,train_input, train_target, return_train_score=True, n_jobs=-1)


print('랜덤포레스트 =                 ',np.mean(scores_rf['train_score']),np.mean(scores_rf['test_score']))
print('엑스트라 트리 =                ',np.mean(scores_et['train_score']),np.mean(scores_et['test_score']))
print('그라디언트 부스트 =            ',np.mean(scores_gb['train_score']),np.mean(scores_gb['test_score']))
print('히스토그램 그라디언트 부스트 = ',np.mean(scores_hgb['train_score']),np.mean(scores_hgb['test_score']))
print('XGBoost =                      ',np.mean(scores_xgb['train_score']),np.mean(scores_xgb['test_score']))
print('LightGBM =                     ',np.mean(scores_lgb['train_score']),np.mean(scores_lgb['test_score']))

print('-------------------------------------------------------')

rf.fit(train_input,train_target)
et.fit(train_input,train_target)
gb.fit(train_input,train_target)
hgb.fit(train_input,train_target)
xgb.fit(train_input,train_target)
lgb.fit(train_input,train_target)



print('랜덤포레스트 =                 ',rf.feature_importances_)
print('-------------------------------------------------------')
print('엑스트라 트리 =                ',et.feature_importances_)
print('-------------------------------------------------------')
print('그라디언트 부스트 =            ',gb.feature_importances_)
print('-------------------------------------------------------')
print('히스토그램 그라디언트 부스트 = ',hgb.score(test_input,test_target))
print('-------------------------------------------------------')
print('XGBoost =                      ',xgb.feature_importances_)
# print(lgb.feature_importances_)

랜덤포레스트 =                  1.0 0.9800000000000001
엑스트라 트리 =                 1.0 0.9800000000000001
그라디언트 부스트 =             1.0 0.9589473684210527
히스토그램 그라디언트 부스트 =  1.0 0.9394736842105263
XGBoost =                       1.0 0.9494736842105264
LightGBM =                      1.0 0.9294736842105262
-------------------------------------------------------
랜덤포레스트 =                  [0.25797416 0.01796373 0.02576245 0.03770346 0.09345008 0.02400221
 0.12840886 0.01058505 0.01141028 0.13119712 0.01034972 0.0185842
 0.23260868]
-------------------------------------------------------
엑스트라 트리 =                 [0.1962303  0.03211827 0.02784845 0.03858197 0.06318405 0.06798185
 0.09340209 0.02839892 0.02447009 0.13508022 0.02039693 0.0327477
 0.23955915]
-------------------------------------------------------
그라디언트 부스트 =             [ 4.09833781e-03  1.90173530e-02  0.00000000e+00  8.95889886e-05
  7.33191038e-03  0.00000000e+00 -4.80816527e-18  0.00000000e+00
  0.00000000e+00  8.57878049e-02  1.7